In [13]:
import requests
import pandas as pd
import glob
import os

In [14]:
# DISTRITAL - solo Lima Metropolitana
for distrito in range(1,44):
    url = f"https://api.resultadoserm2022.onpe.gob.pe/results/03/1401{distrito:02d}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Mobile Safari/537.36",
    }

    resp = requests.get(url, headers=headers)

    data = resp.json()['results']

    fecha = resp.json()['generals']['actData']['FECHA']
    hora = resp.json()['generals']['actData']['HORA']
    avance = resp.json()['generals']['actData']['POR_AVANCE']

    if data != []:
        df = pd.DataFrame(data)
        df['cod_distrito'] = f'{distrito:02d}'
        df['fecha'] = fecha
        df['hora'] = hora
        df['avance'] = avance
        df = df[~(df['C_CODI_AGP'].isin(['80','81']))]
        df = df.dropna(subset=['C_CODI_AGP'])
        df.to_csv(f"resultados_distritales_lima/1401{distrito:02d}.csv")


In [15]:
# setting the path for joining multiple files
files = os.path.join("resultados_distritales_lima", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

li = []

for filename in files:
    df = pd.read_csv(filename, index_col=None, header=0, converters = {'cod_distrito': str.strip, 'avance': str.strip})
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True).drop(columns=['Unnamed: 0'])
frame = frame.sort_values(by=['cod_distrito', 'POR_VALIDOS'])
frame = frame.reset_index(drop=True)

dict_distritos_lima = {
    "01":"LIMA",
    "02":"ANCÓN",
    "03":"ATE",
    "04":"BREÑA",
    "05":"CARABAYLLO",
    "06":"COMAS",
    "07":"CHACLACAYO",
    "08":"CHORRILLOS",
    "09":"LA VICTORIA",
    "10":"LA MOLINA",
    "11":"LINCE",
    "12":"LURIGANCHO",
    "13":"LURÍN",
    "14":"MAGDALENA DEL MAR",
    "15":"MIRAFLORES",
    "16":"PACHACAMAC",
    "17":"PUEBLO LIBRE",
    "18":"PUCUSANA",
    "19":"PUENTE PIEDRA",
    "20":"PUNTA HERMOSA",
    "21":"PUNTA NEGRA",
    "22":"RÍMAC",
    "23":"SAN BARTOLO",
    "24":"SAN ISIDRO",
    "25":"BARRANCO",
    "26":"SAN MARTIN DE PORRES",
    "27":"SAN MIGUEL",
    "28":"SANTA MARIA DEL MAR",
    "29":"SANTA ROSA",
    "30":"SANTIAGO DE SURCO",
    "31":"SURQUILLO",
    "32":"VILLA MARIA DEL TRIUNFO",
    "33":"JESUS MARIA",
    "34":"INDEPENDENCIA",
    "35":"EL AGUSTINO",
    "36":"SAN JUAN DE MIRAFLORES",
    "37":"SAN JUAN DE LURIGANCHO",
    "38":"SAN LUIS",
    "39":"CIENEGUILLA",
    "40":"SAN BORJA",
    "41":"VILLA EL SALVADOR",
    "42":"LOS OLIVOS",
    "43":"SANTA ANITA",
}

frame['distrito'] = frame['cod_distrito'].map(dict_distritos_lima)


Resultant CSV after joining all CSV files at a particular location...


In [16]:
# agregando nombres de candidatos
df_candidatos = pd.read_csv('../candidatos/solo_alcaldes_distritales_lima_metropolitana.csv')
df_candidatos


dict_tildes = {
    "ANCON":"ANCÓN",
    "LURIN":"LURÍN",
    "RIMAC":"RÍMAC",
}
# dict_correcciones_partidos = {
#     "VICTORIA AMAZONENSE":"MOVIMIENTO REGIONAL VICTORIA AMAZONENSE",
#     "ALIANZA GOBIERNO UNIDAD Y ACCION":"ALIANZA GOBIERNO UNIDAD Y ACCION - AGUA",
#     "HATARIY APURIMAC":"MOVIMIENTO REGIONAL HATARIY APURIMAC",
#     "TRABAJO MAS TRABAJO":"MOVIMIENTO REGIONAL TRABAJO MAS TRABAJO",
#     "MOVIMIENTO CIVICO PERUANO":"NUEVA ESPERANZA",
#     "MOVIMENTO INDEPENDIENTE REGIONAL FUERZA TACNA":"MOVIMIENTO INDEPENDIENTE REGIONAL FUERZA TACNA"
# }
df_candidatos['postulaDistrito'] = df_candidatos['postulaDistrito'].replace(dict_tildes)
# df_candidatos['organizacionPolitica'] = df_candidatos['organizacionPolitica'].replace(dict_correcciones_partidos)


df_final = pd.merge(frame, df_candidatos[['postulaDistrito', 'organizacionPolitica', 'nombreCompleto']],  how='left', left_on=['distrito','AGRUPACION'], right_on = ['postulaDistrito','organizacionPolitica'])
df_final = df_final[df_final['distrito'] != 'LIMA']
df_final.to_csv('resultados_distrital_lima.csv')